In [1]:
#!pip install --upgrade pika

In [2]:
from billiard import current_process
import time
import math
import socket
import numpy as np
import os
import random

In [3]:
import pika
pika.__version__

'1.1.0'

In [4]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [5]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [6]:
!sudo rabbitmqctl list_bindings

Listing bindings ...


In [7]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_pings	direct


In [8]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
exchangeName = 'direct_pings'

In [9]:
def calculate_message(msg):
    
    print('msg:' ,msg)
    print('time: ', str(time.time()))
    
    process_name=current_process()._name
    print('process name: ', process_name)
    
    try:
        process_index=str(current_process().index)
    except:
        process_index='_none_'
    
    print('process index: ', process_index)
    
    tmpip= socket.gethostbyname(socket.gethostname())
    print('ip: ', tmpip)
    print(' os_pid: ',str(os.getpid()))
    
    results='routing_'+tmpip+'_'+str(os.getpid())
    
    return results

In [10]:
def on_request(ch, method, props, body):
    global exchangeName
    global queue_name
    
    msg = str(body) # body is the message to the worker. We want to calculate Fib(n)

    print(" I just received message:", msg)
    
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    response = calculate_message(msg) # Results, Fib(n), calculated
    
    ch.queue_bind(exchange=exchangeName, queue=queue_name,routing_key=response)
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key=props.reply_to, # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=str(response))
    #ch.basic_ack(delivery_tag=method.delivery_tag)
    
    print(" Calculations done. Results sent back to the server")
    print(" *** \n")


In [11]:
channel.exchange_declare(exchange=exchangeName, exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange=exchangeName, queue=queue_name,routing_key='ping_queue')

#channel.basic_qos(prefetch_count=1)


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

In [12]:
#When the worker receives a message it will tun the "on_request" call back function
channel.basic_consume(queue=queue_name, on_message_callback=on_request, auto_ack=True)  

'ctag1.080d2343b08a4e71967235925d8bfc07'

In [ ]:
print(" [x] Awaiting requests")
channel.start_consuming()

 [x] Awaiting requests
